# Interactive Demo for Metrics

* command line executables: see README.md
* algorithm documentation: [metrics.py API & Algorithm Documentation](metrics.py_API_Documentation.ipynb)

...some modules and settings for this demo:

In [21]:
from evo.tools import log
log.configure_logging()

In [22]:
from evo.tools import plot
from evo.tools.plot import PlotMode
from evo.core.metrics import PoseRelation, Unit
from evo.tools.settings import SETTINGS

# temporarily override some package settings
SETTINGS.plot_figsize = [6, 6]
SETTINGS.plot_split = True
SETTINGS.plot_usetex = False

# magic plot configuration
import matplotlib.pyplot as plt
%matplotlib inline
%matplotlib notebook

In [23]:
# interactive widgets configuration
import ipywidgets

check_opts_ape = {"align": False, "align_origin": False, "correct_scale": False, "show_plot": True}
check_boxes_ape=[ipywidgets.Checkbox(description=desc, value=val) for desc, val in check_opts_ape.items()]
check_opts_rpe = {"align": False, "align_origin": False, "correct_scale": False, "all_pairs": False, "show_plot": True}
check_boxes_rpe=[ipywidgets.Checkbox(description=desc, value=val) for desc, val in check_opts_rpe.items()]
delta_input = ipywidgets.FloatText(value=1.0, description='delta', disabled=False, color='black')
delta_unit_selector=ipywidgets.Dropdown(
    options={u.value: u for u in Unit if u is not Unit.seconds},
    value=Unit.frames, description='delta_unit'
)
plotmode_selector=ipywidgets.Dropdown(
    options={p.value: p for p in PlotMode},
    value=PlotMode.xy, description='plot_mode'
)
pose_relation_selector=ipywidgets.Dropdown(
    options={p.value: p for p in PoseRelation},
    value=PoseRelation.translation_part, description='pose_relation'
)

---

## Load trajectories

In [24]:
from evo.tools import file_interface
from evo.core import sync

**Load a ROS bagfile** with `geometry_msgs/PoseStamped`, `geometry_msgs/TransformStamped`, `geometry_msgs/PoseWithCovarianceStamped` or `nav_msgs/Odometry` topics:

In [6]:
try:
    import rosbag
    bag_handle = rosbag.Bag("../wheel_slam/bags/MuSe/MuSe_w_RTAB_EKF/s2-s2.bag")
    traj_RTAB_odom = file_interface.read_bag_trajectory(bag_handle, "/rtabmap/odom")
    traj_RTAB_map = file_interface.read_bag_trajectory(bag_handle, "/rtabmap/localization_pose")
    
    traj_wheel_odom = file_interface.read_bag_trajectory(bag_handle, "/odom")
    traj_EKF = file_interface.read_bag_trajectory(bag_handle, "/robot_pose_ekf/odom_combined")
    traj_gt = file_interface.read_bag_trajectory(bag_handle, "/vicon/kobuki_vicon_link/kobuki_vicon_link")
    
except ImportError as e:
    print(e)  # ROS not found

**Load a MuSe_w_clearpathEKF**

In [41]:
try:
    import rosbag
    bag_handle = rosbag.Bag("../wheel_slam/bags/MuSe/MuSe_W_RTAB_clearpathEKF/s2-s2-clearpathEKF-short.bag")
    traj_RTAB_odom = file_interface.read_bag_trajectory(bag_handle, "/rtabmap/vo_odom")
    #traj_RTAB_map = file_interface.read_bag_trajectory(bag_handle, "/rtabmap/localization_pose")
    
    traj_wheel_odom = file_interface.read_bag_trajectory(bag_handle, "/odom")
    traj_EKF = file_interface.read_bag_trajectory(bag_handle, "/odometry/filtered")
    traj_gt = file_interface.read_bag_trajectory(bag_handle, "/vicon/kobuki_vicon_link/kobuki_vicon_link")
    
except ImportError as e:
    print(e)  # ROS not found

In [30]:
try:
    import rosbag
    bag_handle = rosbag.Bag("../wheel_slam/bags/MuSe/MuSe_W_RTAB_clearpathEKF/s2-s2-clearpathEKF-long.bag")
    traj_RTAB_odom = file_interface.read_bag_trajectory(bag_handle, "/rtabmap/vo_odom")
    traj_RTAB_map = file_interface.read_bag_trajectory(bag_handle, "/rtabmap/localization_pose")
    
    traj_wheel_odom = file_interface.read_bag_trajectory(bag_handle, "/odom")
    traj_EKF = file_interface.read_bag_trajectory(bag_handle, "/odometry/filtered")
    traj_gt = file_interface.read_bag_trajectory(bag_handle, "/vicon/kobuki_vicon_link/kobuki_vicon_link")
    
except ImportError as e:
    print(e)  # ROS not found

In [44]:
traj_ref, traj_est = sync.associate_trajectories(traj_gt, traj_RTAB_odom) # traj_ref, traj_est
#_, traj_est = sync.associate_trajectories(traj_gt, traj_RTAB_map) # traj_ref, traj_est
#_, traj_est = sync.associate_trajectories(traj_gt, traj_wheel_odom) # traj_ref, traj_est
#_, traj_est = sync.associate_trajectories(traj_gt, traj_EKF) # traj_ref, traj_est

print(traj_ref)
print(traj_est)

1607 poses, 17.507m path length, 182.402s duration
1607 poses, 16.961m path length, 182.401s duration


## APE

Algorithm and API explanation: [see here](metrics.py_API_Documentation.ipynb#ape_math)


### Interactive APE Demo
***Run the code below, configure the parameters in the GUI and press the update button.***

(uses the trajectories loaded above)

In [40]:
import evo.main_ape as main_ape
import evo.common_ape_rpe as common

count = 0
results = []

def callback_ape(pose_relation, align, align_origin, correct_scale, plot_mode, show_plot):
    global results, count
    est_name="APE Test #{}".format(count)
    
    
    result = main_ape.ape(traj_ref, traj_est, est_name=est_name,
                          pose_relation=pose_relation, align=align, correct_scale=correct_scale,
                          align_origin=align_origin)
    count += 1
    results.append(result)
    print(result.info)
    
    if show_plot:
        fig = plt.figure()
        ax = plot.prepare_axis(fig, plot_mode)
        plot.traj(ax, plot_mode, traj_ref, style="--", alpha=0.5)
        plot.traj_colormap(
            ax, result.trajectories[est_name], result.np_arrays["error_array"], plot_mode,
            min_map=result.stats["min"], max_map=result.stats["max"])
    
_ = ipywidgets.interact_manual(callback_ape, pose_relation=pose_relation_selector, plot_mode=plotmode_selector,**{c.description: c.value for c in check_boxes_ape})

interactive(children=(Dropdown(description='pose_relation', options={'full transformation': <PoseRelation.full…

---

## RPE

Algorithm and API explanation: [see here](metrics.py_API_Documentation.ipynb#rpe_math)

### Interactive RPE Demo

***Run the code below, configure the parameters in the GUI and press the update button.***

(uses the trajectories loaded above, alignment only useful for visualization here)

In [35]:
import evo.main_rpe as main_rpe

count = 0
results = []

def callback_rpe(pose_relation, delta, delta_unit, all_pairs, align, correct_scale, plot_mode, show_plot):
    global results, count
    est_name="RPE Test #{}".format(count)
    result = main_rpe.rpe(traj_ref, traj_est, est_name=est_name,
                          pose_relation=pose_relation, delta=delta, delta_unit=delta_unit, 
                          all_pairs=all_pairs, align=align, correct_scale=correct_scale, 
                          support_loop=True)
    count += 1
    results.append(result)
    
    if show_plot:
        fig = plt.figure()
        ax = plot.prepare_axis(fig, plot_mode)
        plot.traj(ax, plot_mode, traj_ref, style="--", alpha=0.5)
        plot.traj_colormap(
            ax, result.trajectories[est_name], result.np_arrays["error_array"], plot_mode,
            min_map=result.stats["min"], max_map=result.stats["max"])

_ = ipywidgets.interact_manual(callback_rpe, pose_relation=pose_relation_selector, plot_mode=plotmode_selector, 
                               delta=delta_input, delta_unit=delta_unit_selector, 
                               **{c.description: c.value for c in check_boxes_rpe})

interactive(children=(Dropdown(description='pose_relation', options={'full transformation': <PoseRelation.full…

Do stuff with the result objects:

In [63]:
import pandas as pd
from evo.tools import pandas_bridge

df = pd.DataFrame()
for result in results:
    df = pd.concat((df, pandas_bridge.result_to_df(result)), axis="columns")
df

APE Test #0
info      title                          APE w.r.t. full transformation (unit-less)\n(w...
          ref_name                                                               reference
          est_name                                                             APE Test #0
          label                                                            APE (unit-less)
stats     rmse                                                                    0.254818
          mean                                                                    0.230947
          median                                                                  0.199887
          std                                                                     0.107683
          min                                                                     0.111857
          max                                                                     0.497646
          sse                                                                       5.2595
np_arrays error_array                    [0.11907134493178788, 0.11386259052226931, 0.1...
          seconds_from_start             [0.0, 4.406501054763794, 9.677489757537842, 13...
          timestamps                     [1544025522.9455829, 1544025527.352084, 154402...
          alignment_transformation_sim3  [[0.9845137413426976, 0.17449703217859844, -0....

In [64]:
df.loc["stats"]

,APE Test #0
rmse,0.254818
mean,0.230947
median,0.199887
std,0.107683
min,0.111857
max,0.497646
sse,5.2595
